<a href="https://colab.research.google.com/github/Caleb-Flegel/CS2---Code-Example/blob/master/Two_Slit_Dim_Light.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Partner: Caleb Flegel & Zach Washburn

Date: 10/04/2022

# Objectives

# Install and import libraries

In [1]:
!pip install whit-phys-util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import whit_phys_util as buc

# Connect to GitHub

In [3]:
repo = buc.local_repository("https://github.com/PS251-FA22/two-slit-dim-light-1b")

Repository exists at this location. Pulling changes from GitHub.


# Equipment

| Description | Manufacturer | Model | ID or SN |
| ----------- | ------------ | ----- | -------- |
| Two-slit interference apparatus | TeachSpin | TWS 2-A | TWS 383 |
| T-Shaped Card | N/A | N/A | N/A |
| Multimeter  | Fluke | 179 | 58870154 |
| Oscilloscope | Siglent | SDS 1052DL | SDS100P4141674 |
| Digital Pulse Counter | TeachSpin | PCIT-1d | CT0171 |

# Locating New Interference Landmarks

In [4]:
centerLocation = 4.24 #mm
slitBlockerLocation = 5.62 #mm
distanceToDetector = 485 #mm

# Initial Observations of Photon Detections

## Oscilloscope Settings
* Vertical scale for CH1: 50 mV/div
* Horizontal scale: 10 ns/div
* Coupling for CH1: DC
* Trigger: edge, rising, CH1
* Trigger Level: 104 mV

## Observations of Pulses

### PMT Voltage at 840 V
* Amplitude: ~150 mV
* Width: ~6.0 ns

### PMT Voltage at 900V
* Amplitude: ~160 mV 
* Width: ~6.0ns

The aplitute of the pulses appear to be 10 mV higher at 900V than 840V with the width of the pulses staying around 6.0ns.

# Optimizing signal to noise ratio

| PMT voltage (V) | Counts with shutter open | Counts with shutter closed | Signal to noise ratio |
| --- | --- | --- | --- |
| 750 | 691 | 36 | 19.2:1 |
| 800 | 8573 | 124 | 69.1:1 |
| 850 | 18533 | 305 | 60.7:1 |
| 900 | 23146 | 528 | 43.8:1 | 

In [13]:
df1 = pd.read_excel("Two Slit Dim Data.xlsx")
df1['x (mm)'] = df1['Location (mm)']-centerLocation
df1['Count uncertainty'] = 2*np.sqrt(df1['PMT Counts'])
df1

,Location (mm),PMT Counts,x (mm),Count uncertainty
0,2.54,659,-1.70,51.341991
1,2.59,662,-1.65,51.458721
2,2.64,579,-1.60,48.124838
3,2.69,548,-1.55,46.818800
4,2.74,701,-1.50,52.952809
...,...,...,...,...
64,5.74,4289,1.50,130.980915
65,5.79,3761,1.55,122.653985
66,5.84,2736,1.60,104.613575
67,5.89,2341,1.65,96.767763


# Analysis

In [14]:
def I_ts(x, I0, wavelength, x0, dI):
  d = 0.406 #mm
  q = np.arctan((x-x0)/distanceToDetector)
  I2s = I0*np.sinc(a*np.sin(q)/wavelength)**2*np.cos(np.pi*d*np.sin(q)/wavelength)**2
  return I2s + dI